In [1]:
# This is a sample Python script.
import mediapipe as mp
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
import numpy as np

import csv
import scipy as sp      

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.dpi'] = 100

In [3]:
def getAngles(coords):
    u = coords[5,:]-coords[0,:]
    u = u/np.linalg.norm(u)
    a = coords[6,:]-coords[5,:]
    a = a/np.linalg.norm(a)
    angle_index = np.arccos(np.dot(a, u))
    b = coords[7,:]-coords[6,:]
    b = b/np.linalg.norm(b)
    angle_index_2 = np.arccos(np.dot(b, a))
    c = coords[8,:]-coords[7,:]
    c = c/np.linalg.norm(c)
    angle_index_3 = np.arccos(np.dot(c, b))  
    b = a

    u = coords[9,:]-coords[0,:]
    u = u/np.linalg.norm(u)
    a = coords[10,:]-coords[9,:]
    a = a/np.linalg.norm(a)
    angle_middle = np.arccos(np.dot(a, u))  
    angle_index_middle = np.arccos(np.dot(b,a))      
    b = coords[11,:]-coords[10,:]
    b = b/np.linalg.norm(b)
    angle_middle_2 = np.arccos(np.dot(b, a))   
    c = coords[12,:]-coords[11,:]
    c = c/np.linalg.norm(c)
    angle_middle_3 = np.arccos(np.dot(c, b))      
    b=a

    u = coords[13,:]-coords[0,:]
    u = u/np.linalg.norm(u)
    a = coords[14,:]-coords[13,:]
    a = a/np.linalg.norm(a)
    angle_ring = np.arccos(np.dot(a, u)) 
    angle_middle_ring = np.arccos(np.dot(b,a))     
    b = coords[15,:]-coords[14,:]
    b = b/np.linalg.norm(b)
    angle_ring_2 = np.arccos(np.dot(b, a))    
    c = coords[16,:]-coords[15,:]
    c = c/np.linalg.norm(c)
    angle_ring_3 = np.arccos(np.dot(c, b))       
    b=a
    
    u = coords[17,:]-coords[0,:]
    u = u/np.linalg.norm(u)    
    a = coords[18,:]-coords[17,:]
    a = a/np.linalg.norm(a)
    angle_pinky = np.arccos(np.dot(a, u))
    angle_ring_pinky = np.arccos(np.dot(b,a))     
    b = coords[19,:]-coords[18,:]
    b = b/np.linalg.norm(b)
    angle_pinky_2 = np.arccos(np.dot(b, a))    
    c = coords[20,:]-coords[19,:]
    c = c/np.linalg.norm(c)
    angle_pinky_3 = np.arccos(np.dot(c, b))
    
    return np.array([angle_index,angle_index_2,angle_index_3,angle_index_middle,
                     angle_middle,angle_middle_2,angle_middle_3,angle_middle_ring,
                     angle_ring,angle_ring_2,angle_ring_3,angle_ring_pinky,
                     angle_pinky,angle_pinky_2,angle_pinky_3])
def getDistances(coord):
    dist = np.zeros(coord.shape[0]-1)
    for j in range(coord.shape[0]-1):
        dist[j] = np.linalg.norm(coord[j+1,:]-coord[0,:])
    dist = dist/np.sum(dist)  
    
    return dist

def distAnalysis(data):
    distances = np.zeros((data.shape[0], data.shape[1]-1))
    for i in range(data.shape[0]):
        coord = np.array(data[i,:,:])
        distances[i,:] = getDistances(coord)
    mean = np.mean(distances, axis=0)
    std = np.std(distances, axis=0)
    return distances, mean, std

def sortOutData(data, dist, mean, std, sigma_mult=1):
    delidx = []
    for i in range(dist.shape[0]):
        diff = np.abs(dist[i,:]-mean)
        for j in range(diff.shape[0]):
            if diff[j] > sigma_mult*std[j]:
                delidx.append(i)
                break;
    _data = np.delete(data,delidx,0)
    return _data

def writeCSV(all_data):
    with open(output_path + '/data.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=';')        
        for idx,d in enumerate(all_data):
            for i in range(d.shape[0]):
                write_data = np.zeros(3*d.shape[1] + 1)
                write_data[0] = idx
                write_data[1:] = np.array(d[i,:,:].flatten())
                writer.writerow(write_data)
    csvfile.close()

In [4]:
root = tk.Tk()
root.withdraw()
CLASSES = []
IMAGE_FILES = []


ALL_DATA = []
DATA_DISTANCES = []
MEANS = []
STD_DEV = []


# First file dialog to select Input directory (Testdata)
file_path = filedialog.askdirectory()
print('Input: ' + file_path)

# Create list of every subdirectory in selected Test data directory representing different classes
for file in os.listdir(file_path):
    if os.path.isdir(file_path + '/' + file):
        CLASSES.append(file)
print('Classes: ', CLASSES)

# Second file dialog to select the Output folder for this program        
output_path = filedialog.askdirectory()
print('Output: ' + output_path)
        

### Analyse for each class (one folder is a class i.e. "A")

    

for idxC, folder in enumerate(CLASSES):
    sub_path = os.path.join(file_path + '/' + folder)
    IMAGE_FILES.clear()

    # get list of Image_Files
    for file in os.listdir(sub_path):
        if file.endswith('.png') or file.endswith('.jpg'):
            IMAGE_FILES.append(file)    

    print(folder +': ')
    print(IMAGE_FILES)

    color_1  = np.random.rand(3)*0.7+0.15 # for display purposes only: generato random color for plots

    coords = []

    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=1,
        min_detection_confidence=0.6) as hands:

        # Go through each picture of class
        for idx, file in enumerate(IMAGE_FILES):
            ########################################################################################################
            # Preprocessing
            ########################################################################################################

            # flip input image so that a picture of a right hand is detected as a right hand 
            image = cv2.flip(cv2.imread(sub_path + '/' +file),1)
            # cv2 reads image in BGR, image has to be converted to RGB (first tests brought conclusion that hand isn't detected in BGR spektrum)
            image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

            ########################################################################################################
            # Feature Extraction
            ########################################################################################################

            # apply MediaPipe Hands to the picture
            results = hands.process(image)
            # Print handedness and draw hand landmarks on the image.
            if not results.multi_hand_landmarks:
                print(file + ': No hand detected!')
                continue

            # process every hand detected in an image
            for hand_landmarks in results.multi_hand_landmarks:
                write_data = np.zeros(3*21 + 1)
                write_data[0] = idxC

                c = np.zeros((21,3))

                for i in range(21):
                    lm = np.array([hand_landmarks.landmark[i].x,hand_landmarks.landmark[i].y,hand_landmarks.landmark[i].z], dtype=float)
                    c[i,:] = lm
                # raw coordinates of 21 keypoints
                coords.append(c)

                #write_data[1:] = np.array(c.flatten())
                #writer.writerow(write_data)


                #####################################################################################################
                # Feature Analysis
                # This section is purely data calculation to be able to estimate the distribution of key points and angles 
                # to estimate with what amount the 3d keypoint coordinates can be augmented to "still represent" a certain pose
                ######################################################################################################

                # calculate specific joint angles out of the coordinates using vector geometry
                #angles.append(getAngles(coord))
                # calculate the distance between each key-point and the origin (=wrist)

    if len(coords)>0:
        coords_vec = np.stack(coords)
        print(coords_vec.shape)
        # get Distances, Mean Distance and Standard Deviation
        d,m,s = distAnalysis(coords_vec)
        
        # sort out data based on distance and standard deviation
        ALL_DATA.append(sortOutData(coords_vec, d,m,s,sigma_mult=5))
        print('Amount of raw data selected: ' +  str(len(ALL_DATA[-1])) +'/' + str(coords_vec.shape[0]))
    
    
    """
    The following was used to check statistical data analysis 
    """
    
    """
    ax[0].hist(d[:,15], int(d.shape[0]/2), label='Test', color = color_1*0.75)
    ax[0].plot
    plt.show()
    """
    """
    mean_dist = np.mean(dist_vec, axis=0)
    x_dist = np.linspace(0,mean_dist.shape[0]-1, mean_dist.shape[0])
    std_dist = np.std(dist_vec, axis=0)
    #print(std_dist.shape)
    mean_angle = np.mean(angle_vec, axis=0)
    x_angle = np.linspace(0,mean_angle.shape[0]-1, mean_angle.shape[0])
    std_angle = np.std(angle_vec, axis=0)
    ax[0].set_title(folder + ': Angles')
    ax[1].set_title(folder + ': Distances')
    for j in range(dist_vec.shape[0]):
        ax[0].plot(x_angle, angle_vec[j,:],':',linewidth=0.5, color = color_1)
        ax[1].plot(x_dist, dist_vec[j,:],':',linewidth=0.5, color = color_1)
    ax[0].plot(mean_angle, 'o-', label='Mean', linewidth=3, color = color_1*0.75)
    ax[0].errorbar(x_angle, mean_angle, yerr=std_angle, color=color_1*0.5)
    ax[0].set_xlabel('Angle #')
    ax[0].set_ylabel('Joint Angle [°]')
    ax[1].plot(mean_dist, 'o-', label='Mean', linewidth=3, color = color_1*0.75)
    ax[1].errorbar(x_dist, mean_dist, yerr=std_dist, color=color_1*0.5)    
    ax[1].set_xlabel('Keypoint #')
    ax[1].set_ylabel('Normalised Distance to Origin')
    plt.savefig(output_path + '/Diagrams/' + folder + '.png',transparent=True)
    """

# Write list of data as .csv file
writeCSV(ALL_DATA)

    

#plt.show()

Input: C:/Users/gianl/Desktop/val_set
Classes:  ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']
Output: C:/Users/gianl/Desktop/val_set
A: 
['A20221029_210219.png', 'A20221029_210220.png', 'A20221029_210222.png', 'A20221029_210224.png', 'A20221029_210225.png', 'A20221029_210227.png', 'A20221029_210228.png', 'A20221029_210230.png', 'A20221029_210232.png', 'A20221029_210234.png', 'A20221029_210236.png', 'A20221029_210237.png', 'A20221029_210238.png', 'A20221029_210241.png', 'A20221029_210243.png', 'A20221029_210245.png', 'A20221029_210246.png', 'A20221029_210248.png', 'A20221029_210252.png', 'A20221029_210254.png', 'A20221029_210301.png', 'A20221029_210302.png', 'A20221029_210303.png', 'A20221029_210305.png', 'A20221029_210306.png', 'A20221029_210307.png', 'A20221029_210309.png', 'A20221029_210312.png', 'A20221029_210313.png', 'A20221029_210314.png', 'A20221029_210316.png', 'A20221029_210318.png', 'A20221029_210323.p

(42, 21, 3)
Amount of raw data selected: 41/42
H: 
['H20221029_211642.png', 'H20221029_211644.png', 'H20221029_211645.png', 'H20221029_211647.png', 'H20221029_211650.png', 'H20221029_211651.png', 'H20221029_211652.png', 'H20221029_211653.png', 'H20221029_211655.png', 'H20221029_211658.png', 'H20221029_211702.png', 'H20221029_211704.png', 'H20221029_211712.png', 'H20221029_211714.png', 'H20221029_211723.png', 'H20221029_211739.png', 'H20221029_211740.png', 'H20221029_211741.png', 'H20221029_211750.png', 'H20221029_211751.png', 'H20221029_211753.png', 'H20221029_211806.png', 'H20221029_211810.png', 'H20221029_211811.png', 'H20221029_211812.png', 'H20221029_211813.png', 'H20221029_211815.png', 'H20221029_211817.png', 'H20221029_211821.png', 'H20221029_211825.png', 'H20221029_211828.png', 'H20221029_211835.png', 'H20221029_211837.png', 'H20221029_211839.png', 'H20221029_211841.png', 'H20221029_211844.png', 'H20221029_211846.png', 'H20221029_211848.png', 'H20221029_211849.png', 'H20221029_2

P20221029_213327.png: No hand detected!
(40, 21, 3)
Amount of raw data selected: 40/40
Q: 
['Q20221029_213518.png', 'Q20221029_213519.png', 'Q20221029_213522.png', 'Q20221029_213523.png', 'Q20221029_213524.png', 'Q20221029_213525.png', 'Q20221029_213527.png', 'Q20221029_213529.png', 'Q20221029_213532.png', 'Q20221029_213533.png', 'Q20221029_213535.png', 'Q20221029_213537.png', 'Q20221029_213538.png', 'Q20221029_213540.png', 'Q20221029_213542.png', 'Q20221029_213543.png', 'Q20221029_213544.png', 'Q20221029_213546.png', 'Q20221029_213547.png', 'Q20221029_213549.png', 'Q20221029_213603.png', 'Q20221029_213604.png', 'Q20221029_213605.png', 'Q20221029_213606.png', 'Q20221029_213607.png', 'Q20221029_213608.png', 'Q20221029_213610.png', 'Q20221029_213611.png', 'Q20221029_213613.png', 'Q20221029_213616.png', 'Q20221029_213618.png', 'Q20221029_213620.png', 'Q20221029_213622.png', 'Q20221029_213623.png', 'Q20221029_213625.png', 'Q20221029_213628.png', 'Q20221029_213629.png', 'Q20221029_213631.pn

W20221029_214735.png: No hand detected!
(50, 21, 3)
Amount of raw data selected: 50/50
X: 
['X20221029_214904.png', 'X20221029_214905.png', 'X20221029_214907.png', 'X20221029_214908.png', 'X20221029_214909.png', 'X20221029_214910.png', 'X20221029_214912.png', 'X20221029_214913.png', 'X20221029_214914.png', 'X20221029_214916.png', 'X20221029_214917.png', 'X20221029_214919.png', 'X20221029_214920.png', 'X20221029_214921.png', 'X20221029_214922.png', 'X20221029_214923.png', 'X20221029_214924.png', 'X20221029_214925.png', 'X20221029_214926.png', 'X20221029_214927.png', 'X20221029_214929.png', 'X20221029_214930.png', 'X20221029_215024.png', 'X20221029_215025.png', 'X20221029_215027.png', 'X20221029_215028.png', 'X20221029_215029.png', 'X20221029_215030.png', 'X20221029_215032.png', 'X20221029_215034.png', 'X20221029_215035.png', 'X20221029_215037.png', 'X20221029_215038.png', 'X20221029_215039.png', 'X20221029_215040.png', 'X20221029_215041.png', 'X20221029_215043.png', 'X20221029_215044.pn